# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

![table](https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/7JXaz3NNEeiMwApe4i-fLg_40e690ae0e927abda2d4bde7d94ed133_Screen-Shot-2018-06-18-at-7.17.57-PM.png?expiry=1577491200000&hmac=M9BoPKD71JVSyP079GhXaS3ClccJFaqYC3xUFUUYf28)


In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
postal_codes_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
res = requests.get(postal_codes_url)
soup = BeautifulSoup(res.content, 'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [3]:
df.rename(columns={'Postcode': 'PostalCode', 'Neighbourhood': 'Neighborhood'}, inplace=True)
df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [4]:
df_boroughs = df[df['Borough'] != 'Not assigned']

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.


In [5]:
df_agg = (df_boroughs.groupby('PostalCode')
          .agg({'Borough': 'first', 'Neighborhood': ', '.join})
          .reset_index()
          .reindex(columns=df.columns))
df_agg

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.


In [6]:
df_agg['Neighborhood'] = df_agg.apply(lambda x: x['Borough'] if x['Neighborhood'] == 'Not assigned' else x['Neighborhood'], axis=1)

In [7]:
print(df_agg.to_string())

    PostalCode           Borough                                       Neighborhood
0          M1B       Scarborough                                     Rouge, Malvern
1          M1C       Scarborough             Highland Creek, Rouge Hill, Port Union
2          M1E       Scarborough                  Guildwood, Morningside, West Hill
3          M1G       Scarborough                                             Woburn
4          M1H       Scarborough                                          Cedarbrae
5          M1J       Scarborough                                Scarborough Village
6          M1K       Scarborough        East Birchmount Park, Ionview, Kennedy Park
7          M1L       Scarborough                    Clairlea, Golden Mile, Oakridge
8          M1M       Scarborough    Cliffcrest, Cliffside, Scarborough Village West
9          M1N       Scarborough                        Birch Cliff, Cliffside West
10         M1P       Scarborough  Dorset Park, Scarborough Town Centre, Wexf

In [8]:
df_agg.shape

(103, 3)

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:


```python
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
```


In [9]:
assert False # Following code deosn't work since geocode isn't responding

import geocoder
from tqdm import tqdm

df_with_coordinates = df_agg
lattitude = []
longitude = []
for postal_code in tqdm(df_with_coordinates['PostalCode']):
    lat_lng_coords = None
    while lat_lng_coords is None:
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitute.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])

AssertionError: 

Alternatively, we can download coordinates and use them directly

In [10]:
import requests
from io import StringIO

url = "http://cocl.us/Geospatial_data"
csv = requests.get(url)
coordinates = pd.read_csv(StringIO(csv.text))
coordinates

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [11]:
df_with_coordinates = pd.merge(df_agg, coordinates, left_on='PostalCode', right_on='Postal Code')
df_with_coordinates.drop(columns=['Postal Code'], inplace=True)

In [12]:
df_with_coordinates

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


## Visualizing Toronto neighborhoods

Let's create a map of Toronto with neighborhoods

In [14]:
import folium

In [18]:
map_toronto = folium.Map(location=['43.6532', '-79.3832'], zoom_start=10)
df = df_with_coordinates
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood'], df['Borough']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto